# Week 3
Depending on what seems more relevant in your graph, pick one of the following local notions:
- Decay centrality
- Betweeness centrality
- Closeness centrality
- Clustering
- Any other notions that you invent
1) Provide a code computing the given centrality using basic functions of networkx (you are not allowed touse directly nx."what you want").
2) Discuss why you picked this measure and who is the most central in your network based on yourchoice.
3) Provide the cumulative distribution for this centrality and give a graphical representation of your graph(log-log, log or normal representation as you think it is more relevant).